In [1]:
import boto3
import pandas as pd
import psycopg2
import json

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('./cluster.config'))

In [3]:
KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE = config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE = config.get('DWH','DWH_NODE_TYPE')

DWH_CLUSTER_IDENTIFIER = config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB = config.get('DWH','DWH_DB')
DWH_DB_USER = config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH','DWH_PORT')

DWH_IAM_ROLE_NAME = config.get('DWH','DWH_IAM_ROLE_NAME')

In [4]:
pd.DataFrame({'Param':
                 ['DWH_CLUSTER_TYPE', 'DWH_NUM_NODES', 'DWH_NODE_TYPE', 'DWH_CLUSTER_IDENTIFIER', 'DWH_DB', 'DWH_DB_USER', 'DWH_DB_PASSWORD', 'DWH_PORT', 'DWH_IAM_ROLE_NAME'],
             'Value':
                 [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-redshift-cluster-1
4,DWH_DB,dev
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Password1
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift-s3-access


## Service EC2

In [5]:
ec2 = boto3.resource('ec2',
                        region_name='us-west-1',
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                    )

## Services S3, IAM, Redshift

In [6]:
s3 = boto3.resource('s3',
                        region_name='us-west-1',
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                    )

iam = boto3.client('iam',
                        region_name='us-west-1',
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                    )

redshift = boto3.client('redshift',
                        region_name='us-west-1',
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET
                    )

## Connect to S3 bucket

In [7]:
bucket = s3.Bucket('juan-rds-bucket-test')
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'venue_pipe.txt']

## Create cluster

In [8]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [9]:
try:
    response = redshift.create_cluster(
    ClusterType = DWH_CLUSTER_TYPE,
    NodeType = DWH_NODE_TYPE,
    
    # Identifiers and Credentials
    DBName = DWH_DB,
    ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
    MasterUsername = DWH_DB_USER,
    MasterUserPassword = DWH_DB_PASSWORD,
    
    # Roles (for s3 access)
    IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists


## Describe cluster

In [12]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

{'ClusterIdentifier': 'my-redshift-cluster-1',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'available',
 'ClusterAvailabilityStatus': 'Available',
 'MasterUsername': 'awsuser',
 'DBName': 'dev',
 'Endpoint': {'Address': 'my-redshift-cluster-1.cuazxwqypxhf.us-west-1.redshift.amazonaws.com',
  'Port': 5439},
 'ClusterCreateTime': datetime.datetime(2022, 11, 2, 2, 7, 35, 936000, tzinfo=tzutc()),
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-01110c9149ad23775',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-0725a61b90e943a46',
 'AvailabilityZone': 'us-west-1a',
 'PreferredMaintenanceWindow': 'tue:12:30-tue:13:00',
 'PendingModifiedValues': {},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 1,
 'PubliclyAccessi

In [14]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ['ClusterIdentifier', 'NodeType', 'ClusterStatus', 'MasterUsername', 'DBName', 'Endpoint', 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=['key', 'value'])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

/var/folders/nc/_cn5zvxd3615vbm8z4c101rc0000gn/T/ipykernel_1964/832006832.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,key,value
0,ClusterIdentifier,my-redshift-cluster-1
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,dev
5,Endpoint,"{'Address': 'my-redshift-cluster-1.cuazxwqypxhf.us-west-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-0725a61b90e943a46


In [17]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [18]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName = defaultSg.group_name,
        CidrIp = '0.0.0.0/0',
        IpProtocol = 'TCP',
        FromPort = int(DWH_PORT),
        ToPort = int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-01110c9149ad23775')
